In [ ]:
import requests
import json
import pandas as pd
import datetime
from collections import OrderedDict
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from settings import long, lat

Api only allows max of 10k rows per call. For 15 min time intervals there are a load of 0s for rainfall so it doesnt give much data

# Return nearest rainfall monitoring station and set up to collect data

In [ ]:
def rainfall_near_user_input(latitude,longitude,distance):
    #returns details of the water stations near by
    base_url = "http://environment.data.gov.uk/flood-monitoring/id/stations"
    input_lat = str(latitude)
    input_long = str(longitude)
    distance = str(distance)
    user_parameters = '?parameter=rainfall'+'&lat='+input_lat+'&long='+input_long+'&dist='+distance
    station_url = base_url+user_parameters
    get_data = requests.get(station_url)
    get_data.raise_for_status()
    analysis = get_data.json()
    items1 = analysis["items"]
    url_for_station = items1[0]["@id"]
    return(url_for_station)

In [ ]:
dist=10
rain_station = rainfall_near_user_input(lat,long,dist)
#print(rain_station)

In [ ]:
#calc date and determine how much data to bring back
today = datetime.date.today()

#create a list of dates adapted from this link:
#https://www.matthealy.com.au/blog/post/find-months-between-two-dates-with-python/


start = datetime.date(year=today.year -3, month=today.month, day=today.day)
end = today

# Get list of months >= start and < end
months = OrderedDict(((start + datetime.timedelta(_)).strftime("%Y-%m-01"),0) for _ in range((end - start).days))
#convert dictionary to a list
months=list(months.keys())
print(months)

In [ ]:
#create a blank list to save results
rain_data=[]

#iterate through each month in the time period to extract data
for date in tqdm(months):
    #start month = key from dictionary / value in the list
    d1=datetime.datetime.strptime(date,"%Y-%m-%d").date()
    #end month - month after
    d2=datetime.date(year=d1.year + int(d1.month / 12), month=((d1.month % 12) + 1), day=1)
    
    rain_station_url = rain_station +"/readings?_sorted"+"&startdate="+str(d1)+"&enddate="+str(d2)+"&_limit=10000"
    get_data_rain = requests.get(rain_station_url)
    get_data_rain.raise_for_status()
    analysis_rain = get_data_rain.json()
    items_rain = analysis_rain["items"]
    print("start:"+str(d1)+" end:"+str(d2))
    for x in items_rain:
        rain_data.append(x)

In [ ]:
# create a dateframe from the data
rain_df = pd.DataFrame(rain_data)
rain_df['measure']=rain_df['measure'].str.rsplit('/', 1).str[1]
rain_df['dt'] =  pd.to_datetime(rain_df['dateTime'])
rain_df= rain_df.drop(['@id','dateTime'], axis=1)
print(rain_df.shape)
rain_df.head()

In [ ]:
# export to csv
rain_df.to_csv('rainfall_recent.csv')